In [9]:
# 1️⃣ Install dependencies
!pip install -U transformers datasets accelerate

import os
import torch
os.environ["WANDB_DISABLED"] = "true"   # Disable W&B logging

# 2️⃣ Imports
from datasets import load_dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)

# 3️⃣ Load dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# 4️⃣ Load model & tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# 5️⃣ Preprocess function (dynamic padding handled by collator)
def preprocess(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(examples["highlights"], max_length=150, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess, batched=True)

# 6️⃣ Data collator for dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 7️⃣ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,   # Small for Colab
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500
)

# 8️⃣ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(2000)),
    eval_dataset=tokenized_datasets["validation"].shuffle(seed=42).select(range(500)),
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 9️⃣ Train
trainer.train()

# 🔟 Save fine-tuned model
model.save_pretrained("./summarizer_model")
tokenizer.save_pretrained("./summarizer_model")




Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [11]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 1️⃣ Load saved model & tokenizer
model_path = "./summarizer_model"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

# 2️⃣ Set device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 3️⃣ Test function
def summarize_text(text):
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=100, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

# 4️⃣ Example usage
test_text = """
NASA announced its Artemis mission to return humans to the Moon by 2025.
The program aims to establish a sustainable human presence and prepare for Mars missions.
"""
summary = summarize_text(test_text)
print("\nSummary:", summary)

# 5️⃣ Test with your own input
custom_text = input("\nEnter text to summarize:\n")
print("\nSummary:", summarize_text(custom_text))



Summary: NASA announced its Artemis mission to return humans to the Moon by 2025. The program aims to establish a sustainable human presence.

Enter text to summarize:
More than 58,000 people have been killed since Israel launched the assault in October 2023, according to Palestinian health authorities. Israeli forces have also imposed several total blockades on the territory throughout the war, pushing Gaza’s 2.3 million residents to the brink of starvation.

Summary: More than 58,000 people have been killed since Israel launched the assault in October 2023. Israeli forces have also imposed several total blockades throughout the war.


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load model & tokenizer
tokenizer = T5Tokenizer.from_pretrained("./summarizer_model")
model = T5ForConditionalGeneration.from_pretrained("./summarizer_model")

app = FastAPI()

# Input schema
class TextInput(BaseModel):
    text: str

@app.post("/summarize")
async def summarize(input_data: TextInput):
    inputs = tokenizer("summarize: " + input_data.text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"summary": summary}


In [2]:
%%writefile Summarizer.py
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import T5Tokenizer, T5ForConditionalGeneration
import os

MODEL_PATH = "./summarizer_model"

if not os.path.exists(MODEL_PATH):
    raise RuntimeError(f"Model not found at {MODEL_PATH}! Please train or copy it here first.")

tokenizer = T5Tokenizer.from_pretrained(MODEL_PATH)
model = T5ForConditionalGeneration.from_pretrained(MODEL_PATH)

app = FastAPI()

class TextInput(BaseModel):
    text: str

@app.post("/summarize")
async def summarize(input_data: TextInput):
    inputs = tokenizer("summarize: " + input_data.text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"summary": summary}


Writing Summarizer.py


In [3]:
!pip install fastapi uvicorn pyngrok colabcode


Requested uvicorn from https://files.pythonhosted.org/packages/ef/67/546c35e9fffb585ea0608ba3bdcafe17ae402e304367203d0b08d6c23051/uvicorn-0.13.1-py3-none-any.whl has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^
Please use pip<24.1 if you need to use this version.
INFO: pip is looking at multiple versions of colabcode to determine which version is compatible with other requirements. This could take a while.
  Using cached uvicorn-0.13.1-py3-none-any.whl.metadata (4.6 kB)
Requested uvicorn from https://files.pythonhosted.org/packages/ef/67/546c35e9fffb585ea0608ba3bdcafe17ae402e304367203d0b08d6c23051/uvicorn-0.13.1-py3-none-any.whl has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^
Please use pip<24.1 if you need to use this version.
  Using cached uvicorn-0.13.1-py3-none-any.whl.

In [4]:
!pip install fastapi uvicorn pyngrok colabcode

from colabcode import ColabCode

# This will start FastAPI server using app.py
ColabCode(port=8000, code=False, path="app.py")


TypeError: ColabCode.__init__() got an unexpected keyword argument 'code'

In [6]:
!pip install fastapi uvicorn pyngrok

from pyngrok import ngrok

# 🔹 Add your authtoken
!ngrok config add-authtoken 30Hp3ggHO2VyEkOtL7XXIrnzgBv_2dhqUQVo6XVQzzgpt5McZ

# 🔹 Start tunnel
public_url = ngrok.connect(8000)
print("🚀 Public URL:", public_url)

# 🔹 Run FastAPI app from app.py
!uvicorn app:app --host 0.0.0.0 --port 8000



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Public URL: NgrokTunnel: "https://86a18f7dd470.ngrok-free.app" -> "http://localhost:8000"


2025-07-31 16:09:15.669848: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753978155.696157    4716 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753978155.703831    4716 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-31 16:09:15.730678: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/usr/local/bin/uvicorn", line 10, in <module>
    sys.exit(main())
             ^^